# Asynchronous programming with Python
## Module 3 - AsyncIO

### Agenda:

* Key definitions
* Common patterns

## Key definitions
`asyncio` is a built-in Python library for async concurrency and I/O.
The most basic `asyncio` program looks like following:

```python
import asyncio


async def main():
    print('hello')
    await asyncio.sleep(1)
    print('world')


asyncio.run(main())

```
See it running:

In [ ]:
import asyncio


async def main():
    print('hello')
    await asyncio.sleep(1)
    print('world')

# Normally you run asyncio programs with
# asyncio.run(main())
# Though, the notebook magic works just like this:
await main()

### Event Loop
Event loop is a core of the application, used to schedule tasks,
run I/O operations, execute callbacks.  Also in modern asyncio
programs you should rarely deal with it, it still can be found
more often than you would want.

**Projects created prior to Python 3.7 have it around all the time.**

First, startup instructions often propose this snippet:
```python
loop = asyncio.get_event_loop()
loop.run_until_complete(main())
```

Or even:
```python
event_loop = asyncio.get_event_loop()
try:
    event_loop.run_until_complete(main())
finally:
    event_loop.close()
```

Instead, use this:
```python
asyncio.run(main())
```

Second, there are `loop` arguments and attributes, though they are
often deprecated.

AioHTTP is an example of both cases:
<https://docs.aiohttp.org/en/v3.7.4/client_reference.html#client-session>

**You may use event loop to tune your application.**

For docker images and kubernetes applications you often need to limit
the number of threads.

```python
from concurrent.futures.thread import ThreadPoolExecutor


loop = asyncio.get_running_loop()
loop.set_default_executor(ThreadPoolExecutor(max_workers=MAX_THREAD_WORKERS))
```


Prior to Python 3.9 to run `func` in a thread, one should do:

```python
loop = asyncio.get_running_loop()
loop.run_in_executor(None, func)
```

or:
```python
from functools import partial

loop = asyncio.get_running_loop()
loop.run_in_executor(None, partial(func, **args, **kwargs))
```

Starting from Python 3.9, just do:

```python
asyncio.to_thread(func, *args, **kwargs)
```

### Task
`asyncio` uses tasks to wrap and execute your `async def` functions.

Sometimes you need to create it explicitly with
[asyncio.create_task()](https://docs.python.org/3/library/asyncio-task.html#asyncio.create_task).

For more information on tasks refer to the documentation:

* https://docs.python.org/3/library/asyncio-task.html#task-object

Note, however, that most of the time you'll need only
[asyncio.create_task()](https://docs.python.org/3/library/asyncio-task.html#asyncio.create_task)
and [Task.cancel()](https://docs.python.org/3/library/asyncio-task.html#asyncio.Task.cancel).

## Common patterns

### Collecting tasks
[asyncio.gather()](https://docs.python.org/3/library/asyncio-task.html#asyncio.gather)
allows you to schedule asynchronous tasks and execute them concurrently.

In [ ]:
import asyncio


async def child1():
    print("  child1: started! sleeping now...")
    await asyncio.sleep(1)
    print("  child1: still working...")
    await asyncio.sleep(1)
    print("  child1: exiting!")


async def child2():
    print("  child2: started! sleeping now...")
    await asyncio.sleep(2)
    print("  child2: still working...")
    await asyncio.sleep(2)
    print("  child2: exiting!")


async def main():
    tasks = [asyncio.create_task(c) for c in [child1(), child2()]]
    return await asyncio.gather(*tasks)


try:
    print(await main())
except Exception as err:
    # If you want to break one of the children, this code will help
    # to see the whole picture.
    print("The main task failed with", repr(err), "let's see what happens now.")
    await asyncio.sleep(5)
    raise

##### Some practice

**1. If one of the children raises an error, what happens to the other child?
What happens to the main task?**
**1.1. Change the situation with `asyncio.gather(*tasks, return_exceptions=True)`**
**1.2. Cancel the tasks instead, when one of the children raises an exception.**

#### Mapping of results
The order of the results provided with `asyncio.gather` corresponds
to the order of tasks provided.

We can utilize it to map gathered results to input values, or to
map them to other gathered results.

In [ ]:
import asyncio
import random


async def get_double(n):
    print(f"    { n }: thinking on a double...")
    await asyncio.sleep(random.random())
    print(f"    { n }: got it: { n * 2 }")
    return n * 2


async def main():
    # Prepare some arbitrary ordered numbers
    numbers = list(range(10))
    random.shuffle(numbers)

    print("Processing numbers:", numbers)
    tasks = [asyncio.create_task(get_double(n)) for n in numbers]
    result = await asyncio.gather(*tasks)
    return dict(zip(numbers, result))


result = await main()
print(result)
assert result == {n: n*2 for n in range(10)}

##### Some practice

**1. Add `get_triple` async function that returns passed value multiplied by `3`.
     Make `main` return a map of doubles to triples.**
**1.1. Can you collect both doubles and triples concurrently,
       within the same `asyncio.gather`?**

Similar to [asyncio.gather()](https://docs.python.org/3/library/asyncio-task.html#asyncio.gather),
there are also functions [asyncio.wait()](https://docs.python.org/3/library/asyncio-task.html#asyncio.wait)
and [asyncio.as_completed()](https://docs.python.org/3/library/asyncio-task.html#asyncio.as_completed).

They are quite handy if you do not want to wait for all
asynchronous tasks to finish, also
[asyncio.as_completed()](https://docs.python.org/3/library/asyncio-task.html#asyncio.as_completed)
is more clean of them both.

In [ ]:
import asyncio
import random


async def get_double(n):
    print(f"    { n }: thinking on a double...")
    await asyncio.sleep(random.random())
    print(f"    { n }: got it: { n * 2 }")
    return n, n * 2


async def main():
    # Prepare some arbitrary ordered numbers
    numbers = list(range(10))
    random.shuffle(numbers)

    print("Processing numbers:", numbers)
    tasks = [asyncio.create_task(get_double(n)) for n in numbers]

    result = {}
    for coro in asyncio.as_completed(tasks):
        key, value = await coro
        result[key] = value

    return result


result = await main()
print(result)
assert result == {n: n*2 for n in range(10)}


### Dealing with blocking code
Here, `factorial` is contains some heavy-computation code,
defined with `async def` by accident.

What would you see when the code is executed?

In [ ]:
import asyncio
import time


async def tick():
    start = time.time()
    print("    tick started")
    await asyncio.sleep(1)
    print("    tick continued at", time.time() - start)
    await asyncio.sleep(1)
    print("    tick ended at", time.time() - start)



async def factorial(num):
    start = time.time()
    print("Processing the factorial of", num)
    if num < 1:
        raise ValueError("The number should be ≥ 1")
    result = 1
    for i in range(1, num+1):
        result *= i

    print("The factorial of", num, "is", result)
    print("The processing took", time.time() - start)
    return result


loop = asyncio.get_running_loop()


_, factorial_value = await asyncio.gather(
    asyncio.create_task(tick()),
    asyncio.create_task(factorial(100000)),
)

factorial_value

👉 *`factorial` is a toy example.  Also, there is plenty of code
that would block your application.  One of the examples is
[regular expressions](https://medium.com/ochrona/python-dos-prevention-the-redos-attack-7267a8fa2d5c).
Another is validation with [jsonschema](https://github.com/Julian/jsonschema/issues/277).*👈

##### Some practice

**1. Place `asyncio.sleep(0)` inside the `factorial` function to unblock the application.**
**2. Use `loop.run_in_executor()` as provided above to unblock the application.
     Note that `loop.run_in_executor()` accepts synchronous functions, not coroutines.**

👉*Three useful implementations of `asyncio.sleep()`:
(1) make your code wait;
(2) insert a checkpoint to your synchronous code;
(3) quickly create an awaitable object with a ready-to-use result with
`asyncio.sleep(0, result=<anything>)`.*👈

👉*As noted above, to execute some code in a thread with Python 3.9+
simply use [asyncio.to_thread()](https://docs.python.org/3/library/asyncio-task.html#asyncio.to_thread)*👈

### Producers and consumers

In [ ]:
import asyncio
import random


async def main():
    queue = asyncio.Queue(maxsize=1)

    consumer_task = asyncio.create_task(consumer(queue))
    producer_task = asyncio.create_task(producer(queue))

    await producer_task
    await queue.join()

    consumer_task.cancel()
    print("Finished")


async def producer(queue: asyncio.Queue):
    for i in range(3):
        print(f"    sending a message: {i}")
        await queue.put(f"message {i}")


async def consumer(queue: asyncio.Queue):
    while True:
        value = await queue.get()
        print(f"    got value {value!r}")

        # Simulate hard working
        await asyncio.sleep(random.random())

        print(f"    finished processing {value!r}")
        queue.task_done()


try:
    await main()
except Exception as err:
    # If you want to break one of the children, this code will help
    # to see the whole picture.
    print("The main task failed with", repr(err), "let's see what happens now.")
    await asyncio.sleep(5)
    raise

##### Some practice
**1. Add more producers and consumers.  Concurrently await for producers to finish.**

**2. What happens if consumers intermittently fail before `queue.task_done()` is called?
     Try adding following snippet:**
```python
        if random.random() > 0.5:
            print("going to fail now")
            raise ValueError(value)
```

**Note that if consumers fail, both `await asyncio.gather(*producer_task)`
and `await queue.join()` may block.  If there is no backpressure,
(queue maxsize limit), only `await queue.join()` would block, also producers
would keep filling the queue with their data.**

To unblock the code, do following:

1. Wrap blocking statement with
[asyncio.wait_for()](https://docs.python.org/3/library/asyncio-task.html#asyncio.wait_for)
and [asyncio.shield()](https://docs.python.org/3/library/asyncio-task.html#asyncio.shield).
2. If `asyncio.TimeoutError` occurred, check for failed consumers
with [asyncio.wait()](https://docs.python.org/3/library/asyncio-task.html#asyncio.wait).

Here is an example:

In [ ]:
import asyncio
import random
from typing import List, Awaitable


async def main():
    queue = asyncio.Queue(maxsize=1)

    consumer_tasks = [asyncio.create_task(consumer(tid, queue)) for tid in range(3)]
    producer_tasks = [asyncio.create_task(producer(tid, queue)) for tid in range(3)]

    try:
        await supervise(
            consumer_tasks,
            asyncio.gather(queue.join(), *producer_tasks)
        )
    finally:
        for task in consumer_tasks:
            task.cancel()

    print("Finished")


async def supervise(consumers: List[asyncio.Task], checkpoint: Awaitable, timeout=1):
    while True:
        try:
            await asyncio.wait_for(asyncio.shield(checkpoint), timeout=timeout)
        except asyncio.TimeoutError:
            done, consumers = await asyncio.wait(
                consumers, timeout=timeout, return_when=asyncio.FIRST_EXCEPTION,
            )
            # If there is an exception in consumer, it will be raised here:
            await asyncio.gather(*done)
        else:
            break


async def producer(tid, queue: asyncio.Queue):
    for i in range(3):
        print(f"    {tid}: sending a message: {tid}{i}")
        await queue.put(f"message {tid}{i}")


async def consumer(tid, queue: asyncio.Queue):
    while True:
        value = await queue.get()

        # Simulate hard working
        await asyncio.sleep(random.random())

        print(f"    {tid}: got value {value!r}")
        queue.task_done()


try:
    await main()
except Exception as err:
    # If you want to break one of the children, this code will help
    # to see the whole picture.
    print("The main task failed with", repr(err), "let's see what happens now.")
    await asyncio.sleep(5)
    raise

### Conclusion
`asyncio` is a quite robust library for async concurrency and I/O.
It is a standard library, it has well-developed ecosystem.

Though, you should take care to avoid common issues,
such as orphan tasks.

<span style="font-size: x-large">Add your code below:</span>